In [1]:
# !git clone https://github.com/wasef-c/EmoSpeech.git


In [2]:
# !pip install torch transformers[torch] datasets scikit-learn matplotlib seaborn 

In [3]:
# !pip install --upgrade pillow
# 

In [4]:
# !pip install natten

In [5]:
# !python3 -c "import torch; print(torch.__version__)"

In [6]:
# !pip3 install natten==0.17.3+torch210cu118 -f https://shi-labs.com/natten/wheels

In [7]:
# !pip install sentencepiece
# !pip install googletrans


In [ ]:
## Standard libraries
#BEST SO FAR: 20250128_14
import os
import logging
import warnings
import random
from collections import Counter
import math

# PyTorch
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from datasets import load_dataset
from sklearn.utils import resample

# Hugging Face Transformers
from transformers import (
    AutoImageProcessor,
    DinatForImageClassification,
    TrainingArguments,
    get_scheduler,
    BertModel,
    AutoTokenizer,
)
# Hugging Face Datasets
from datasets import load_dataset, concatenate_datasets

# Data processing and metrics
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

# Progress bar
from tqdm.auto import tqdm
best_epoch = 0
# Custom functions (from your own module)
from functionsV3 import *
import numpy as np
# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
label_mapping = {
    0: 'C',
    1: 'N',
    2: 'H',
    3: 'S',
    4: 'U',
    5: 'F',
    6: 'A',
    7: 'D'
}
#{0: 'C', 1: 'N', 2: 'H', 3: 'S', 4: 'U', 5: 'F', 6: 'A', 7: 'D'}
# Directories and Model Config
base_dir = r"./DinatMEL"
output_dir = create_unique_output_dir(base_dir)
os.makedirs(output_dir, exist_ok=True)

DATASET_PATH = "../data"
# CHECKPOINT_PATH = "./NLPIMG_Model_001"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "shi-labs/dinat-mini-in1k-224"  # For processor loading if needed
checkpoint_path = "./EmoDom/best_model.pt"
# checkpoint_path = "./DinatMEL/20250126_3/best_model.pt"
# checkpoint_path = "./DinatMEL/20250125_1/best_model.pt"
pretrain_model = model_path
bert_model_name = "bert-base-uncased"
BATCH_SIZE = 60

# Load Dataset
dataset_name = "cairocode/MSPP_MEL_6"
dataset = load_dataset(dataset_name)

unique_labels = set(dataset['train']['EmoClass'])
print("Unique labels in dataset:", unique_labels)


# Reverse the mapping for efficient lookup
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Define a filtering function
def is_mappable(example):
    return example["EmoClass"] in reverse_label_mapping

column  = "label"

# Filter out rows with unmappable values
filtered_dataset = dataset.filter(is_mappable)

# Map the filtered dataset to add the new column
filtered_dataset = filtered_dataset.map(lambda x: {"label": reverse_label_mapping[x["EmoClass"]]})


# Define a filtering function
def is_valid_transcript(example):
    return isinstance(example['transcript'], str)


# Filter out non-string transcripts in each split
filtered_dataset = filtered_dataset.filter(is_valid_transcript)

inv_label_mapping = {v: k for k, v in label_mapping.items()}

# Step 2: Specify an order for the numeric labels
ordered_labels_numeric = [0, 1, 2, 3, 4, 5, 6, 7 ]#, 7]
ordered_labels_str = [label_mapping[i] for i in ordered_labels_numeric]

# all_test_labels = [...]
# all_test_predictions = [...]

# Step 3: Generate the confusion matrix

train_test_split = filtered_dataset['train'].train_test_split(test_size=0.2, seed=42)

train_val_split = train_test_split['train'].train_test_split(test_size=0.2, seed=42)

train_dataset = train_val_split['train']
# Assuming the labels are stored in the 'label' column
unique_labels = set(train_dataset['label'])
print("Unique labels in train_dataset:", unique_labels)
# Sort the train dataset by the 'label' column


sorted_train = train_dataset.sort("label")
label_boundaries = get_label_boundaries(sorted_train, "label")
print(label_boundaries)
# Print the class distribution before balancing using label boundaries
# Initialize the count array with zeros
count = np.zeros((8,), dtype=int)

print("Class distribution before balancing:")

# Assuming label_boundaries is a dictionary like {0: (0, 50), 1: (51, 100), ...}
for lbl, (start, end) in label_boundaries.items():
    count[lbl] = end - start
    print(f"Label {lbl}: {count[lbl]} samples")

# Calculate the average target size per class
avg_count = int(np.mean(count))
print("Per-label target size:", avg_count)


# Balance the dataset
balanced_train_dataset = balance_dataset_by_sort(
    sorted_train,
    label_boundaries,
    avg_count,
    label_column="label",
    seed=42
)
# balanced_train_dataset = train_dataset

# # Compute new label boundaries for the balanced dataset
# balanced_label_boundaries = get_label_boundaries(
#     balanced_train_dataset.sort("label"),
#     "label"
# )

# # Print the class distribution after balancing
# print("\nClass distribution after balancing:")
# for lbl, (start, end) in balanced_label_boundaries.items():
#     count = end - start
#     print(f"Label {lbl}: {count} samples")

# # Final size of the balanced dataset
# print("\nFinal balanced training size:", len(balanced_train_dataset))



val_dataset = train_val_split['test']
sorted_val = val_dataset.sort("label")
label_boundaries = get_label_boundaries(sorted_val, "label")
print(label_boundaries)
# Print the class distribution before balancing using label boundaries
# Initialize the count array with zeros
count = np.zeros((8,), dtype=int)

print("Class distribution before balancing:")

# Assuming label_boundaries is a dictionary like {0: (0, 50), 1: (51, 100), ...}
for lbl, (start, end) in label_boundaries.items():
    count[lbl] = end - start
    # print(f"Label {lbl}: {count[lbl]} samples")

# Calculate the average target size per class
avg_count = int(np.mean(count))
print("Per-label target size:", avg_count)


# Balance the dataset
balanced_val_dataset = balance_dataset_by_sort(
    sorted_val,
    label_boundaries,
    avg_count,
    label_column="label",
    seed=42
)

val_dataset = balanced_val_dataset

test_dataset = train_test_split['test']
num_labels = 8
print("Test dataset size:", len(test_dataset))
print("Train dataset size:", len(train_dataset))

class_weights = calculate_class_weights(
    train_dataset)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
# class_weoghts = torch(ones)
# Transforms (assumed imported from functions_old)
balanced_train_dataset.set_transform(train_transforms)
train_dataset.set_transform(train_transforms)
val_dataset.set_transform(val_transforms)
test_dataset.set_transform(val_transforms)

train_sampler = CustomSampler(train_dataset)

train_loader = DataLoader(
    balanced_train_dataset,
    # sampler=train_sampler,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=lambda examples: collate_fn_reg(examples, column=column),
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda examples: collate_fn_reg(examples, column=column),
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda examples: collate_fn_reg(examples, column=column),
)


# Load Models
image_model = DinatForImageClassification.from_pretrained(
    pretrain_model,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
    problem_type="single_label_classification",
).to(device)

processor = DinatForImageClassification.from_pretrained(model_path).to(device)

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name).to(device)


# ----------------------------------------------------------------------
# Initialize the Combined Regression Model
# ----------------------------------------------------------------------
unfozen_layers = [10,11]
next_layer_to_unfreeze = unfozen_layers[0]-1
 
model = CombinedModelsBi(
    image_model=image_model,
    bert_model=bert_model,
    image_feature_dim=512,
    bert_embedding_dim=768,
    combined_dim=512,
    num_labels=num_labels,
    unfrozen_layers = unfozen_layers

).to(device)

if checkpoint_path != None:
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # # Define the keywords to include and exclude
    include_keyword = "model"
    exclude_keys = {
        "image_model.classifier.weight",
        "image_model.classifier.bias",
        "fc3"
    }

    # Use dictionary comprehension to filter the keys
    filtered_checkpoint = {
        key: value for key, value in checkpoint.items()
        if include_keyword in key and key not in exclude_keys
    }
    # Load the filtered state dict
    # filtered_checkpoint = checkpoint
    model.load_state_dict(filtered_checkpoint, strict=False)


# model.load_state_dict(torch.load("/media/carol/Data/Documents/Emo_rec/Notebooks/DINAT_BERT/MSPP_COMP/20250105_11/best_model.pt"))
#Mapping of categories to integers: {'D': 0, 'H': 1, 'A': 2, 'O': 3, 'N': 4, 'C': 5, 'F': 6, 'S': 7}
#Mapping of categories to integers: {'F': 0, 'O': 1, 'H': 2, 'D': 3, 'A': 4, 'S': 5, 'N': 6, 'C': 7}
# Training Arguments
training_args = TrainingArguments(
    output_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=50,
    weight_decay=0.01,
    load_best_model_at_end=True
)

logging.getLogger().addHandler(logging.NullHandler())
logging.getLogger("natten.functional").setLevel(logging.ERROR)

# Optimizer & Scheduler
optimizer = optim.AdamW(
    model.parameters(),
    lr=training_args.learning_rate,
    weight_decay=training_args.weight_decay
)

num_training_steps = len(train_loader) * training_args.num_train_epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=num_training_steps,
)

focal_loss = AdaptiveLearnableFocalLoss(class_weights=class_weights)
focal_loss = AdaptiveLearnableFocalLoss()

focal_loss = AdaptiveLearnableFocalLoss_V2(num_classes = num_labels)
# cecc_loss= CrossEntropyWithContrastiveCenterLoss(
#     num_classes=8,  # Number of emotion classes
#     feature_dim=512,  # Feature dimension from fc2
#     alpha=1,  # Adjust weights for CE vs Contrastive-Center
#     beta=0.2
# )
#1.0, beta=0.01
cecc_loss= BalancedCrossEntropyWithContrastiveLoss(
    num_classes=8,  # Number of emotion classes
    feature_dim=512,  # Feature dimension from fc2
    alpha=1.5,  # Adjust weights for CE vs Contrastive-Center
    beta=0.02,
    gamma = 1
)


# BalancedCrossEntropyWithContrastiveLoss 0.01

'''
Start with alpha=0.5, beta=1.0, gamma=0.5
If UAR is too low, increase alpha
If STD is too high, increase gamma
If classes are not well-separated, increase beta

alpha=0.5, beta=1.0, gamma=0.5
'''

# Training Variables
num_epochs = training_args.num_train_epochs
patience = 12
best_val_accuracy = 0
patience_counter = 0

train_losses, val_losses, epochs_list = [], [], []

best_model_path = os.path.join(output_dir, "best_model.pt")
class_weights = None
##############################################################################
# Training Loop
##############################################################################
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for batch in progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask
        )
        logits = outputs["logits"]
        features =  outputs["combined_features"]

        loss = cecc_loss(logits,features, labels)
        # loss = focal_loss(logits, labels)

        # focal_loss = AdaptiveLearnableFocalLoss_V2(num_classes = num_labels, class_weights = class_weights)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}")
    lr_scheduler.step()

    # Validation
    model.eval()
    val_loss = 0
    all_predictions, all_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                pixel_values=pixel_values,
                bert_input_ids=input_ids,
                bert_attention_mask=attention_mask
            )
            logits = outputs["logits"]
            features =  outputs["combined_features"]

            loss = cecc_loss(logits,features, labels)

            # loss = focal_loss(logits, labels)
            val_loss += loss.item()

            predictions = torch.argmax(logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = accuracy_score(all_labels, all_predictions)
    uar = recall_score(all_labels, all_predictions, average="macro")
    f1 = f1_score(all_labels, all_predictions, average="macro")
    per_class_recall = recall_score(all_labels, all_predictions, average=None)
    uar_std = np.std(per_class_recall)

    gamma = 1.5
    comparison_metric = uar / (1 + gamma * uar_std)

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    epochs_list.append(epoch + 1)

    print(
        f"Validation Loss: {avg_val_loss:.4f}, "
        f"Accuracy: {accuracy:.4f}, UAR: {uar:.4f}, F1: {f1:.4f}, UAR STD: {uar_std} "
        f"\nPer-class Recall: {per_class_recall}"
        f"\n Comparison metric: {comparison_metric}"

    )
    plot_and_save_confusion_matrix(all_labels, all_predictions, ordered_labels_str, output_dir, epoch=epoch)


    # Early Stopping based on Accuracy
    if comparison_metric > best_val_accuracy:
        best_val_accuracy = comparison_metric
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        torch.save(model.image_model.state_dict(), "fine_tuned_image_model.pth")
        best_epoch = epoch
        print("Validation uar improved. Best model saved.")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
    # epoch_dir = os.path.join(output_dir, f"Epoch_{epoch}_model.pt")
    # torch.save(model.state_dict(), epoch_dir)

    # import torch
    
    # epsilon = 0.1
    # class_weights = [(1 - r) + epsilon for r in per_class_recall]
    # class_weights = torch.tensor(class_weights, dtype=torch.float32)
    
    # print("Computed class_weights:", class_weights)

    if (epoch + 1) % 3 == 0 and next_layer_to_unfreeze >= 0:
            print(f"Unfreezing BERT layer {next_layer_to_unfreeze}")
            unfreeze_bert_layer(model.bert_model, next_layer_to_unfreeze)
            
            next_layer_to_unfreeze -= 1


# Load Best Model
print("Loading best model for final evaluation.")
model.load_state_dict(torch.load(best_model_path))
model.to(device)

##############################################################################
# Test Evaluation
##############################################################################
print("\nStarting Test Evaluation...")
model.eval()
test_loss = 0
all_test_predictions, all_test_labels = [], []

with torch.no_grad():
    test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)
    for batch in test_progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask
        )
        logits = outputs["logits"]

        loss = F.cross_entropy(logits, labels)
        test_loss += loss.item()

        predictions = torch.argmax(logits, dim=-1)
        all_test_predictions.extend(predictions.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
test_accuracy = accuracy_score(all_test_labels, all_test_predictions)
test_uar = recall_score(all_test_labels, all_test_predictions, average="macro")
test_f1 = f1_score(all_test_labels, all_test_predictions, average="macro")

metrics_str = (
    f"Test Loss: {avg_test_loss:.4f}, "
    f"Accuracy: {test_accuracy:.4f}, "
    f"UAR: {test_uar:.4f}, "
    f"F1: {test_f1:.4f}"
)
print(metrics_str)


# Step 1: Define your label mapping
# label_mapping = {'D': 0, 'H': 1, 'A': 2, 'O': 3, 'N': 4, 'C': 5, 'F': 6, 'S': 7}
inv_label_mapping = {v: k for k, v in label_mapping.items()}

# Step 2: Specify an order for the numeric labels
ordered_labels_numeric = [0, 1, 2, 3, 4, 5, 6, 7]
ordered_labels_str = [label_mapping[i] for i in ordered_labels_numeric]

# Suppose these are your test labels and predictions (as numeric):
# all_test_labels = [...]
# all_test_predictions = [...]


# Step 2: Specify an order for the numeric labels
ordered_labels_numeric = [0, 1, 2, 3, 4, 5, 6, 7]
ordered_labels_str = [label_mapping[i] for i in ordered_labels_numeric]

# Suppose these are your test labels and predictions (as numeric):
# all_test_labels = [...]
# all_test_predictions = [...]

# # Step 3: Generate the confusion matrix
# cm = confusion_matrix(
#     y_true=all_test_labels, 
#     y_pred=all_test_predictions, 
#     # labels=ordered_labels_numeric
# )
plot_and_save_confusion_matrix(all_test_labels, all_test_predictions, ordered_labels_str, output_dir, epoch=None)
# # Step 4: Plot the confusion matrix with custom axis labels
# plt.figure(figsize=(8, 6))
# sns.heatmap(
#     cm,
#     annot=True,
#     fmt="d",
#     cmap="Blues",
#     xticklabels=ordered_labels_str,
#     yticklabels=ordered_labels_str
# )
# plt.xlabel("Predicted Labels")
# plt.ylabel("True Labels")
# plt.title("Confusion Matrix")

# # Save and close
# save_path = os.path.join(output_dir, "confusion_matrix.png")
# plt.savefig(save_path, bbox_inches="tight", dpi=300)
# plt.close()
# print(f"Confusion matrix saved to: {save_path}")

# # Save Metadata
# save_training_metadata(
#     output_dir=output_dir,
#     pathstr=pretrain_model,
#     dataset_name=dataset_name,
#     model_type="CombinedModelsDDCA",
#     super_loss_params="N/A",
#     speaker_disentanglement=True,
#     entropy=False,
#     column="label",
#     metrics=metrics_str,
#     weight_decay=training_args.weight_decay,
#     results=metrics_str
# )

# Overall Metrics (if needed across multiple runs):
# For a single run, these will just match the test metrics.
overall_accuracy = test_accuracy
overall_UAR = test_uar
overall_F1 = test_f1
full_accuracy = test_accuracy

# Save final metrics
output_file = os.path.join(output_dir, "metrics.txt")
with open(output_file, "w") as f:
    f.write(f"Overall F1 Score: {overall_F1:.4f}\n")
    f.write(f"Overall Accuracy: {overall_accuracy:.4f}\n")
    f.write(f"Full Accuracy: {full_accuracy:.4f}\n")
    f.write(f"Overall UAR: {overall_UAR:.4f}\n")
    f.write(f" Class Mapping :{label_mapping}\n")
    f.write(f" Best Epoch :{best_epoch}\n")

print(f"Metrics saved to {output_file}")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Unique labels in dataset: {'S', 'U', 'X', 'N', 'A', 'O', 'C', 'D', 'F', 'H'}
Unique labels in train_dataset: {0, 1, 2, 3, 4, 5, 6, 7}
OrderedDict([(0, (0, 2520)), (1, (2520, 25960)), (2, (25960, 40715)), (3, (40715, 46283)), (4, (46283, 48836)), (5, (48836, 49767)), (6, (49767, 57764)), (7, (57764, 58993))])
Class distribution before balancing:
Label 0: 2520 samples
Label 1: 23440 samples
Label 2: 14755 samples
Label 3: 5568 samples
Label 4: 2553 samples
Label 5: 931 samples
Label 6: 7997 samples
Label 7: 1229 samples
Per-label target size: 7374
OrderedDict([(0, (0, 630)), (1, (630, 6523)), (2, (6523, 10176)), (3, (10176, 11532)), (4, (11532, 12130)), (5, (12130, 12369)), (6, (12369, 14412)), (7, (14412, 14749))])
Class distribution before balancing:
Per-label target size: 1843
Test dataset size: 18436
Train dataset size: 58993


Some weights of DinatForImageClassification were not initialized from the model checkpoint at shi-labs/dinat-mini-in1k-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([8, 512]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch 1/50:   0%|          | 0/984 [00:00<?, ?it/s]

Epoch 1/50 - Training Loss: 4.4787
Validation Loss: nan, Accuracy: 0.3030, UAR: 0.3030, F1: 0.2596, UAR STD: 0.2216626007006973 
Per-class Recall: [0.04449267 0.24091156 0.31144872 0.67335865 0.31036354 0.00434075
 0.60499186 0.23440043]
 Comparison metric: 0.2274220721808813
Confusion matrix saved to: ./DinatMEL/20250130_6/Epoch0_val_confusion_matrix.png
Validation uar improved. Best model saved.


Epoch 2/50:   0%|          | 0/984 [00:00<?, ?it/s]

Epoch 2/50 - Training Loss: 4.1963
Validation Loss: nan, Accuracy: 0.3286, UAR: 0.3286, F1: 0.2899, UAR STD: 0.21628275898072583 
Per-class Recall: [0.07487792 0.2794357  0.41888226 0.6863809  0.29408573 0.0168204
 0.59142702 0.26695605]
 Comparison metric: 0.2481140579346619
Confusion matrix saved to: ./DinatMEL/20250130_6/Epoch1_val_confusion_matrix.png
Validation uar improved. Best model saved.


Epoch 3/50:   0%|          | 0/984 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
best_epoch

In [ ]:

# Load Best Model
print("Loading best model for final evaluation.")
model.load_state_dict(torch.load(best_model_path))
model.to(device)

##############################################################################
# Test Evaluation
##############################################################################
print("\nStarting Test Evaluation...")
model.eval()
test_loss = 0
all_test_predictions, all_test_labels = [], []

with torch.no_grad():
    test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)
    for batch in test_progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask
        )
        logits = outputs["logits"]

        loss = F.cross_entropy(logits, labels)
        test_loss += loss.item()

        predictions = torch.argmax(logits, dim=-1)
        all_test_predictions.extend(predictions.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
test_accuracy = accuracy_score(all_test_labels, all_test_predictions)
test_uar = recall_score(all_test_labels, all_test_predictions, average="macro")
test_f1 = f1_score(all_test_labels, all_test_predictions, average="macro")

metrics_str = (
    f"Test Loss: {avg_test_loss:.4f}, "
    f"Accuracy: {test_accuracy:.4f}, "
    f"UAR: {test_uar:.4f}, "
    f"F1: {test_f1:.4f}"
)
print(metrics_str)


# Step 1: Define your label mapping
# label_mapping = {'D': 0, 'H': 1, 'A': 2, 'O': 3, 'N': 4, 'C': 5, 'F': 6, 'S': 7}
inv_label_mapping = {v: k for k, v in label_mapping.items()}

# Step 2: Specify an order for the numeric labels
ordered_labels_numeric = [0, 1, 2, 3, 4, 5, 6, 7]
ordered_labels_str = [label_mapping[i] for i in ordered_labels_numeric]


In [ ]:
plot_and_save_confusion_matrix(all_test_labels, all_test_predictions, ordered_labels_str, output_dir, epoch=None)



In [ ]:

# Step 1: Define your label mapping
# label_mapping = {'D': 0, 'H': 1, 'A': 2, 'O': 3, 'N': 4, 'C': 5, 'F': 6, 'S': 7}
inv_label_mapping = {v: k for k, v in label_mapping.items()}

# Step 2: Specify an order for the numeric labels
ordered_labels_numeric = [0, 1, 2, 3, 4, 5, 6, 7 ]#, 7]
ordered_labels_str = [label_mapping[i] for i in ordered_labels_numeric]

# all_test_labels = [...]
# all_test_predictions = [...]

# # Step 3: Generate the confusion matrix
# cm = confusion_matrix(
#     y_true=all_test_labels, 
#     y_pred=all_test_predictions, 
#     # labels=ordered_labels_numeric
# )

# # Step 4: Plot the confusion matrix with custom axis labels
# plt.figure(figsize=(8, 6))
# sns.heatmap(
#     cm,
#     annot=True,
#     fmt="d",
#     cmap="Blues",
#     xticklabels=ordered_labels_str,
#     yticklabels=ordered_labels_str
# )
# plt.xlabel("Predicted Labels")
# plt.ylabel("True Labels")
# plt.title("Confusion Matrix")

# # Save and close
# save_path = os.path.join(output_dir, "confusion_matrix.png")
# plt.savefig(save_path, bbox_inches="tight", dpi=300)
# plt.close()
# print(f"Confusion matrix saved to: {save_path}")

# # # Save Metadata
# # save_training_metadata(
# #     output_dir=output_dir,
# #     pathstr=pretrain_model,
# #     dataset_name=dataset_name,
# #     model_type="CombinedModelsDDCA",
# #     super_loss_params="N/A",
# #     speaker_disentanglement=True,
# #     entropy=False,
# #     column="label",
# #     metrics=metrics_str,
# #     weight_decay=training_args.weight_decay,
# #     results=metrics_str
# # )

# Overall Metrics (if needed across multiple runs):
# For a single run, these will just match the test metrics.
overall_accuracy = test_accuracy
overall_UAR = test_uar
overall_F1 = test_f1
full_accuracy = test_accuracy

# Save final metrics
output_file = os.path.join(output_dir, "metrics.txt")
with open(output_file, "w") as f:
    f.write(f"Overall F1 Score: {overall_F1:.4f}\n")
    f.write(f"Overall Accuracy: {overall_accuracy:.4f}\n")
    f.write(f"Full Accuracy: {full_accuracy:.4f}\n")
    f.write(f"Overall UAR: {overall_UAR:.4f}\n")
    f.write(f"Class Mapping :{label_mapping}\n")
    f.write(f" Best Epoch :{best_epoch}\n")
    
    f.write(f"--------------PARAMETERS --------------\n")
    f.write(f"Pretrain Model :{pretrain_model}\n")
    f.write(f"Dataset Name: {dataset_name}\n")
    f.write(f"Class Weights :{class_weights}\n")
    f.write(f"Weight Decay: {training_args.weight_decay}\n")
    f.write(f"Model Type  CombinedModelsBi \n")
    

print(f"Metrics saved to {output_file}")

inv_label_mapping

In [ ]:
## from datasets import DatasetDict
from sklearn.model_selection import train_test_split
import csv
from functions_old import *
import datasets
from datasets import load_dataset
import os
import logging
import warnings
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import (
    AutoImageProcessor,
    ViTForImageClassification,
    BertModel,
    AutoTokenizer,
    get_scheduler,
    DinatForImageClassification,
)
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
# Load Best Model
print("Loading best model for final evaluation.")
model.load_state_dict(torch.load(best_model_path))
model.to(device)


def collate_fn_test(examples):
    """
    Custom collate function to handle batching of image data and BERT inputs.
    """
    # print(examples)
    pixel_values = torch.stack([example["pixel_values"]
                               for example in examples]).to(device)
    input_ids = torch.stack([example["input_ids"]
                            for example in examples]).to(device)
    attention_mask = torch.stack(
        [example["attention_mask"] for example in examples]).to(device)
    files = [example["file"] for example in examples]

    bert_embeddings = torch.stack(
        [example["bert_embeddings"] for example in examples]).to(device)

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "files": files,
        "bert_embeddings": bert_embeddings
    }

# checkpoint_path = "./DinatCurriculum/Regression/Valence/20250116_19/best_model.pt"
# output_dir = "./DinatCurriculum/Regression/Valence/20250116_19/
# os.makedirs(output_dir, exist_ok=True)


dataset_name = 'cairocode/MSPP_TEST_MEL_6'
# test_dataset = 
test_dataset = load_dataset(dataset_name)['test']

test_dataset.set_transform(train_transforms)

test_loader = DataLoader(
    test_dataset,
    batch_size=50,
    collate_fn=collate_fn_test,
)


print("\nStarting Test Evaluation...")
model.eval()
test_loss = 0.0
all_test_predictions = []
all_test_labels = []
all_filenames = []  # To store filenames

with torch.no_grad():
    test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)
    for batch in test_progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        # labels = batch["labels"].float().to(device)

        # Extract filenames from the batch (assuming "filename" is part of batch)
        filenames = batch["files"]  # Adjust key as necessary
        all_filenames.extend(filenames)

        outputs_dict = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask
        )
        predictions = outputs_dict["logits"].argmax(dim=-1)  # Convert logits to class indices

        # loss = criterion(predictions, labels)
        # test_loss += loss.item()

        all_test_predictions.extend(predictions.cpu().numpy())
        # all_test_labels.extend(labels.cpu().numpy())


# Correct the mapping logic
# Ensure each prediction is converted to an integer before mapping
mapped_labels = [label_mapping[int(pred)] for pred in all_test_predictions]

output_csv_path = os.path.join(output_dir, "test_predictions.csv")
with open(output_csv_path, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Filename", "Category"])
    for filename, prediction in zip(all_filenames, mapped_labels):
        writer.writerow([filename, prediction])

print(f"Predictions saved to {output_csv_path}")

# print(dataset_dict)
